## Exploratory Analysis using Jupyter Notebook
For further reading, we recommend: 
- [the pandas documentation](http://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html#getting)  for information about using DataFrames
- [this blog post](https://towardsdatascience.com/introduction-to-data-visualization-in-python-89a54c97fbed) for a jumpstart into visualizations
- [the matplotlib documentation](https://matplotlib.org/users/pyplot_tutorial.html) for more info about visualizations

In [1]:
import pandas as pd
import numpy as np

#### Loading data from our GPC bucket

In [57]:
import s3fs
s3 = s3fs.S3FileSystem(anon=True)
base = 'twde-datalab/raw'
for full_file in s3.ls(base):
    file = full_file[len(base)+1::]
    print(full_file)
    if (len(file)):
        s3.get(full_file, '../data/%s'%file)

twde-datalab/raw/
twde-datalab/raw/bigTable.csv
twde-datalab/raw/cities.csv
twde-datalab/raw/cpi.csv
twde-datalab/raw/holidays_events.csv
twde-datalab/raw/items.csv
twde-datalab/raw/last_year_train.csv
twde-datalab/raw/oil.csv
twde-datalab/raw/quito_stores_sample.csv
twde-datalab/raw/quito_stores_sample2016-2017.csv
twde-datalab/raw/sample_submission.csv
twde-datalab/raw/sample_test.csv
twde-datalab/raw/sample_train.csv
twde-datalab/raw/store47-2016.csv
twde-datalab/raw/stores.csv
twde-datalab/raw/test.csv
twde-datalab/raw/train.csv
twde-datalab/raw/train30days.csv
twde-datalab/raw/transactions.csv


In [12]:
import glob
import os

csvs = glob.glob('../data/*.csv')
raw = {}

for file in csvs:
    size = int(os.path.getsize(file)/1024/1024)
    name = file.split('/')[-1].split('.')[0]
    print(name, size, 'MB')
    if size<512:
        print('loading %s'%file)
        raw[name] = pd.read_csv(file)

store47-2016 79 MB
loading ../data/store47-2016.csv
train30days 125 MB
loading ../data/train30days.csv
train 4765 MB
items 0 MB
loading ../data/items.csv
sample_train 0 MB
loading ../data/sample_train.csv
sample_test 0 MB
loading ../data/sample_test.csv
quito_stores_sample2016-2017 340 MB
loading ../data/quito_stores_sample2016-2017.csv
oil 0 MB
loading ../data/oil.csv
sample_submission 38 MB
loading ../data/sample_submission.csv
last_year_train 1479 MB
cpi 0 MB
loading ../data/cpi.csv
cities 0 MB
loading ../data/cities.csv
quito_stores_sample 1328 MB
bigTable 687 MB
transactions 1 MB
loading ../data/transactions.csv
test 120 MB
loading ../data/test.csv
holidays_events 0 MB
loading ../data/holidays_events.csv
stores 0 MB
loading ../data/stores.csv


In [24]:
for key,value in raw.items():
    print('\033[1m\033[91m' + key, str(value.shape) + '\033[0m')
    print(value.head())
    print()

store47-2016 (997231, 15)
   Unnamed: 0        id        date  item_nbr  unit_sales        family  \
0        7808  88219279  2016-08-16    103520        10.0     GROCERY I   
1        7809  88219280  2016-08-16    103665         4.0  BREAD/BAKERY   
2        7810  88219281  2016-08-16    105574         9.0     GROCERY I   
3        7811  88219282  2016-08-16    105575        45.0     GROCERY I   
4        7812  88219283  2016-08-16    105577         8.0     GROCERY I   

   class  perishable  transactions  year  month  day  dayofweek  \
0   1028           0          3570  2016      8   16          1   
1   2712           1          3570  2016      8   16          1   
2   1045           0          3570  2016      8   16          1   
3   1045           0          3570  2016      8   16          1   
4   1045           0          3570  2016      8   16          1   

   days_til_end_of_data  dayoff  
0                   364   False  
1                   364   False  
2                 

#### With just this glimpse, you can start to fill out your list of assumptions, hypotheses, and questions. Some of mine are:
- Question: What is the span of dates we are provided?
- Question: How many distinct store_nbr values are there?
- Question: How many distinct item_nbr values are there?
- Hypothesis: unit_sales are always positive
- Hypothesis: onpromotion is always either True or False
- Hypothesis: city and state are always going to be Quito and Pichincha
- Hypothesis: cluster is always 5
- Question: What does cluster mean and is it important to know?
- Question: How many records does the data contain?
- Question: What other data files are available?

### Here's some examples of how to address those first questions

In [26]:
# Access an entire dataframe column like you would
# the value in a python dictionary:
# (The returned object has similar pandas built-in 
# functions, like 'head' and 'max')
data = raw['store47_sample2016-2017']
date_min = data['date'].min()
date_max = data['date'].max()

In [27]:
# Dataframe columns also have a 'unique' method,
# which can answer several of our questions from above
store_list = data['store_nbr'].unique()
store_target = 47

In [7]:
print(data['item_nbr'].unique())
print("There are too many item numbers to display, so let's just count them for now:")
print("\n{} different item_nbr values in our data"
          .format(len(data['item_nbr'].unique())))

[ 103520  103665  105574 ... 2011468 2011448 2123839]
There are too many item numbers to display, so let's just count them for now:

3717 different item_nbr values in our data


#### It might be helpful to know the 'shape' of our data. We could count by hand (for now) the columns, but how many rows do we have altogether?

In [8]:
print(data.shape)
print("There are {} rows and {} columns in our data".format(data.shape[0], data.shape[1]))

(5877318, 9)
There are 5877318 rows and 9 columns in our data


#### Moving along to answer our intial questions... Let's have a look at unit_sales. Keep in mind that unit sales is the variable we want to predict with our science.

Each row in our data is essentially telling us a `unit_sales` number for a given `item_nbr` at a given `store_nbr` on a given `date`. That is, "how many of an item was sold at a store on a day".

In [16]:
print("unit sales smaller than zero: {}".format(np.sum(data['unit_sales'] < 0)))

unit sales smaller than zero: 477


In [24]:
on_promo = np.sum(data['onpromotion'] == True)
off_promo = np.sum(data['onpromotion'] == False)
print("unit sales on promotion: {}".format(on_promo))
print("unit sales off promotion: {}".format(off_promo))
print("unit neither on/off promotion: {}".format(data.shape[0] - on_promo - off_promo))

unit sales on promotion: 623970
unit sales off promotion: 5253348
unit neither on/off promotion: 0


In [27]:
print('unique states: {}'.format(data['state'].unique()))

unique states: ['Pichincha']
